In [1]:
# Imports
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

c:\Users\austi\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
# Load Data
conn = sqlite3.connect("cfb_data.db")
games_full_df = pd.read_sql_query("SELECT * FROM games_full", conn)
returning_df = pd.read_sql_query("SELECT * FROM returning_production", conn)
conn.close()

print(games_full_df.shape)
games_full_df.head()

(9817, 149)


,id,season,week,season_type,start_date,start_time_tbd,completed,neutral_site,conference_game,attendance,...,away_defense_rushingPlays_successRate,away_defense_rushingPlays_explosiveness,away_defense_passingPlays_ppa,away_defense_passingPlays_totalPPA,away_defense_passingPlays_successRate,away_defense_passingPlays_explosiveness,home_turnovers,home_possessionTime,away_turnovers,away_possessionTime
0,332410006,2013,1,regular,2013-08-29T23:30:00.000Z,0.0,1,1,0,15240.0,...,0.382353,0.588519,0.149096,4.025596,0.370370,1.382872,1,1575.0,0,2025.0
1,332410023,2013,1,regular,2013-08-30T02:00:00.000Z,0.0,1,0,0,13136.0,...,0.315789,0.791780,-0.077537,-1.938417,0.320000,1.288211,0,1432.0,2,2168.0
2,332410041,2013,1,regular,2013-08-29T23:30:00.000Z,0.0,1,0,0,30689.0,...,0.363636,0.843752,0.232041,5.801016,0.400000,1.617091,2,2141.0,1,2183.0
3,332410062,2013,1,regular,2013-08-30T03:00:00.000Z,NaN,1,0,0,39058.0,...,0.208333,0.961625,-0.047574,-1.855368,0.282051,1.443332,4,1565.0,2,2830.0
4,332410084,2013,1,regular,2013-08-29T23:00:00.000Z,0.0,1,0,0,40278.0,...,0.259259,1.149174,0.340396,5.786730,0.411765,1.846431,3,1949.0,2,1662.0


In [3]:
returning_df.head()

,season,team,conference,totalPPA,totalPassingPPA,totalReceivingPPA,totalRushingPPA,percentPPA,percentPassingPPA,percentReceivingPPA,percentRushingPPA,usage,passingUsage,receivingUsage,rushingUsage
0,2014,Air Force,Mountain West,96.7,7.3,60.0,29.4,0.608,0.417,0.780,0.453,0.684,0.751,0.774,0.652
1,2014,Akron,Mid-American,168.8,46.0,121.7,1.2,0.819,1.000,0.772,0.480,0.952,1.000,0.833,0.989
2,2014,Alabama,SEC,197.4,1.9,131.3,64.3,0.574,0.021,0.715,0.915,0.589,0.078,0.779,0.911
3,2014,Arizona,Pac-12,91.4,NaN,92.5,-1.1,0.340,0.000,0.595,-0.018,0.142,0.000,0.619,0.048
4,2014,Arizona State,Pac-12,243.5,82.9,129.3,31.4,0.669,0.953,0.562,0.668,0.713,0.990,0.550,0.562


In [10]:
alabama_games = games_full_df[(games_full_df['home_team'] == 'Alabama') | (games_full_df['away_team'] == 'Alabama')]
alabama_games[['id','season','week','home_team','away_team','home_points','away_points','season_type']].iloc[:25,:]

,id,season,week,home_team,away_team,home_points,away_points,season_type
56,332430259,2013,1,Virginia Tech,Alabama,10.0,35.0,regular
188,332570245,2013,3,Texas A&M,Alabama,42.0,49.0,regular
251,332640333,2013,4,Alabama,Colorado State,31.0,6.0,regular
302,332710333,2013,5,Alabama,Ole Miss,25.0,0.0,regular
353,332780333,2013,6,Alabama,Georgia State,45.0,3.0,regular
382,332850096,2013,7,Kentucky,Alabama,7.0,48.0,regular
458,332920333,2013,8,Alabama,Arkansas,52.0,0.0,regular
505,332990333,2013,9,Alabama,Tennessee,45.0,10.0,regular
613,333130333,2013,11,Alabama,LSU,38.0,17.0,regular
666,333200344,2013,12,Mississippi State,Alabama,7.0,20.0,regular


In [ ]:
home_teams = set(games_full_df.home_team.unique().tolist())
away_teams = set(games_full_df.away_team.unique().tolist())
teams = home_teams | away_teams
teams = list(teams)
for item in teams:
    print(item+",")

In [ ]:
return_teams = returning_df.team.unique().tolist()
for item in return_teams:
    print(item+",")